In [9]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from scipy.spatial import cKDTree
from shapely.geometry import Point
from preparacion_datos.src.preparacion_inter_puntos import prep
import glob
import tqdm as tqdm
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.geometry
import warnings
warnings.filterwarnings("ignore")

### PREDIAL

In [2]:
casas=gpd.read_file(r'C:\Users\dlara\Naucalpan_new_casas.shp')
# terrenos=gpd.read_file(r'C:\Users\dlara\Downloads\naucalpancompleto\new_terreno1.shp')

In [2]:
predial=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIAL (16022022).xlsx')
adeudo=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIALADEUDO (16022022).xlsx')
propietario=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PROPIETARIO (16022022).xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\dlara\\Downloads\\Recepcion de informacion-20230302T185018Z-001\\Recepcion de informacion\\Naucalpan de Juárez_PREDIAL (16022022).xlsx'

In [3]:
base_pred= pd.read_csv(r'C:\Users\dlara\Downloads\Predial.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\dlara\\Downloads\\Predial.csv'

In [9]:
from geoloc2.Valor_Catastral.calculo_predial import *

ImportError: attempted relative import with no known parent package

In [ ]:
curts= gpd.read_file(r'C:\Users\dlara\Downloads\098 Naucalpan\NAUCALPAN_PREDIOS.shp')

In [ ]:
NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')

C:\Users\dlara\AppData\Local\Temp\ipykernel_14656\1341984383.py:1: DtypeWarning: Columns (2,4,5,6,8,9,10,11,12,13,14,15,16,17,19,20,21,23,24,25,26,27,30,32,34,35,38,39,40,43,44,49,50,51,52,53,55,56,57,60,61,62,63,64,65,66,67,68,69,70,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,101,102,103,104,107,108,109,110,111,113,114,115,116,117,118,119,124,127,129,130,131,134,135,137,138,139,140,141,142,146,156,157,158,159,160,161,162,164,166,167,168,169,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,219,220) have mixed types. Specify dtype option on import or set low_memory=False.
  NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')


In [ ]:
def transform_df_to_gpd(df: pd.DataFrame,
                    lon_col: str = 'INTFIS_RS_FD_LON_1',
                    lat_col: str = 'INTFIS_RS_FD_LAT_1', crs= 'EPSG:4326'):
        '''Transforma base a partir de dos columnas de latitud y longitud'''
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                df[lon_col], df[lat_col]
            ),
            crs=crs,
        )
        return gdf

In [ ]:
NAUC['INTFIS_RS_FD_LAT']= NAUC['INTFIS_RS_FD_LAT_1'].combine_first(NAUC['INTFIS_RS_FD_LAT'])
NAUC['INTFIS_RS_FD_LON']= NAUC['INTFIS_RS_FD_LON_1'].combine_first(NAUC['INTFIS_RS_FD_LON'])

In [ ]:
NAUC=transform_df_to_gpd(NAUC, lon_col= 'INTFIS_RS_FD_LON',lat_col = 'INTFIS_RS_FD_LAT')

In [ ]:
PRED= NAUC.loc[NAUC['CLAVECATASTRAL'].notna()]

In [ ]:
Manzanas=gpd.read_file(r'C:\Users\dlara\Downloads\Full manzanas\Full manzanas\Manzana_Naucalpan.shp')

In [ ]:
Manzanas['geometry']=Manzanas.buffer(100)

In [ ]:
Manz=Manzanas.dissolve()

In [ ]:
NAUC=NAUC.to_crs(Manz.crs)
def new_func(NAUC):
    return gpd.sjoin(NAUC,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

rectificado_nauc=new_func(NAUC)

In [ ]:
curts=curts.to_crs(Manz.crs)
curts_rectificado=gpd.sjoin(curts,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

In [ ]:
casas=casas.to_crs(Manz.crs)
casas['ID']=casas.reset_index()['index']
casas_rectificado=gpd.sjoin(casas,Manz).drop(columns=[	'index_right',	'mun',	'zona',	'manz',	'cve_cat'])

## Cruces

In [558]:
def line2poli(BASE):    
    linestring=BASE.loc[(BASE["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(BASE["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()

    ARR=linestring.geometry.astype(str)
    print(ARR)
    if len(ARR)>0:
        # print(ARR)
        
        # print(l1)
        for i in range(len(ARR)):
            try:
                # print(ARR)
                l1=ARR.str.find('POLYGON')
                print(ARR)
                list_wkt = ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",'')
                # print(ARR,list_wkt)
                list_wkt= [list_wkt+str('))')]
                # print(list_wkt)
                list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
                multi=shapely.geometry.MultiPolygon(list_polygons)
                print(multi)
                # print('multi', multi)
                linestring['geometry'] = multi

            except:
                # print(ARR)
                l1=ARR.str.find('LINESTRING')
                # print(ARR.str[:l1])
                list_wkt = ARR.str.replace(ARR.str[:l1],'').str.replace("GEOMETRYCOLLECTION (LINESTRING ",'')
                list_wkt= [list_wkt+str('))')]
                # print(list_wkt)
                list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
                multi=shapely.geometry.MultiPolygon(list_polygons)
                # print('multi', multi)
                linestring['geometry'] = multi
                
            for col in dictt['cata']:
                try:
                    
                    BASE=BASE[~BASE[col].isin(linestring[col])]
                except:
                    pass
            
            BASE = pd.concat([BASE,linestring])

            return(BASE)

In [546]:
linestring=shps_merged.loc[(shps_merged["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(shps_merged["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()

ARR=linestring.geometry.astype(str)
print(ARR)

if len(ARR)>0:
    # print(ARR)
    
    # print(l1)
    for i in range(len(ARR)):
        try:
            # print(ARR)
            l1=ARR.str.find('POLYGON')
            print(ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",''))
            list_wkt = ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",'')
            # print(ARR,list_wkt)
            list_wkt= [list_wkt+str('))')]
            # print(list_wkt)
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
            multi=shapely.geometry.MultiPolygon(list_polygons)
            print(multi)
            # print('multi', multi)
            linestring['geometry'] = multi

        except:
            # print(ARR)
            l1=ARR.str.find('LINESTRING')
            # print(ARR.str[:l1])
            list_wkt = ARR.str.replace(ARR.str[:l1],'').str.replace("GEOMETRYCOLLECTION (LINESTRING ",'')
            list_wkt= [list_wkt+str('))')]
            # print(list_wkt)
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
            multi=shapely.geometry.MultiPolygon(list_polygons)
            # print('multi', multi)
            linestring['geometry'] = multi
            
        for col in dictt['cata']:
            try:
                
                shps_merged=shps_merged[~shps_merged[col].isin(linestring[col])]
            except:
                pass
        
        shps_merged = pd.concat([shps_merged,linestring])

Series([], Name: geometry, dtype: object)


In [423]:
dictt= {'cata':['id_cat','cve_cat','CLAVECATAS','CLAVECATASTRAL_1','CLAVECATASTRAL_2']}
dictt['cata']

['id_cat', 'cve_cat', 'CLAVECATAS', 'CLAVECATASTRAL_1', 'CLAVECATASTRAL_2']

In [552]:

path=r"C:\Users\dlara\Desktop\MUNICIPIOS\Insumos_municipios_en_proceso\*"
filenames= glob.glob(path)
for files in tqdm.tqdm(filenames):

    shapes={}
       
    i=0
    files=files.replace('\\',"/")
    path2=files.replace('\\',"/").replace('/*',"/")
    
    for file in glob.glob(path2 + '/*.shp'):
        file= file.replace('/',"\\")
        name=file.replace(path2.replace('/',"\\"),'').replace('.shp',"").replace('\\',"") 
        i=i+1
        Nname= name + str(i)
    #   
        shapes[Nname]=gpd.read_file(file)
        
        # print(Nname)
        # reemplazar id cat por clave catastral
        
        for col in dictt['cata']:
            shapes[Nname].rename(columns={col:'CLAVECATASTRAL'}, inplace=True)
        #Disolución de las geometrías a partir de clavecatastral 
        shapes[Nname]=shapes[Nname].dissolve(by='CLAVECATASTRAL').reset_index()
        
        if shapes[Nname].crs == 4326:
            pass
        else:
            shapes[Nname]=shapes[Nname].to_crs(4326)

    lista= list(shapes.keys())       
    for i in tqdm.tqdm(range(len(lista))):
        # if  (Nname.casefold() in i.casefold()):
        #     # print(Nname.casefold(), i.casefold())
        #     if Nname.casefold() == i.casefold():
                # print((Nname.casefold(), i.casefold()),)
                # continue
            # else:
        
        shps_merged= shapes[lista[i]]
        shps_merged.crs= 4326
        try:
            shps_merged=shapes[lista[i+1]].overlay(shps_merged, how='union',keep_geom_type=False)
            # print(shps_merged)
            shps_merged= line2poli(shps_merged)

        except:
            # print(shapes[lista[i]].head())
            shps_merged= line2poli(shapes[lista[i]])

            # print(name)
            # print(shps_merged)
            shps_merged.to_file(r'C:/Users/dlara/Desktop/MUNICIPIOS/' + Nname + "_cruce_shapes.shp")
            # else: 
            # print('error')  
            pass

  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\dlara\AppData\Local\Temp\ipykernel_15048\2461850265.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  print(ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",''))
 20%|██        | 1/5 [00:02<00:10,  2.61s/it]

0    GEOMETRYCOLLECTION (POLYGON ((-99.869484 19.78...
Name: geometry, dtype: object
Series([], Name: geometry, dtype: object)
Series([], Name: geometry, dtype: object)


C:\Users\dlara\AppData\Local\Temp\ipykernel_15048\2461850265.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  print(ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",''))


0    GEOMETRYCOLLECTION (POLYGON ((-99.661838 18.82...
Name: geometry, dtype: object
Series([], Name: geometry, dtype: object)


 40%|████      | 2/5 [00:08<00:13,  4.53s/it]

...

0    GEOMETRYCOLLECTION Z (POLYGON Z ((-99.291201 1...
Name: geometry, dtype: object
Series([], Name: geometry, dtype: object)


C:\Users\dlara\AppData\Local\Temp\ipykernel_15048\2461850265.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  print(ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",''))
 80%|████████  | 4/5 [1:24:46<33:24, 2004.47s/it]

Series([], Name: geometry, dtype: object)


100%|██████████| 5/5 [1:24:46<00:00, 1017.35s/it]

Series([], Name: geometry, dtype: object)


In [2]:
PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)
# 
PRED[['CLAVECATASTRAL', 'LONGITUD','LATITUD','CURT_1', 'curt','geometry','min_dist']].dropna()

# prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857).to_crs(4326)

PRED.drop_duplicates('geometry', inplace=True)
PRED.dropna(axis=1, inplace=True)
puntos=prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857)
puntos.dropna(axis=1, inplace=True)
rev_pred=gpd.read_file(r'C:\Users\dlara\Downloads\final\final\test_igecem_final.shp')

UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
nounicos=nounicos.to_crs(3857)
puntos=puntos.set_geometry('geometry')
rev_pred=rev_pred.set_geometry('geometry')
rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
# from preparacion_datos.src.preparacion_inter_puntos import ckdnearest

j_puntos=prep.ckdnearest(puntos, puntos[['geometry']])
cruce_check= gpd.sjoin(nounicos.to_crs(3857),j_puntos.loc[j_puntos['min_dist_2']>30][['geometry', 'LONGITUD_1', 'LATITUD_1','min_dist']], how='right')

cruce_check.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_reps.shp')
UNICOS.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_unicos.shp')

C:\Users\dlara\AppData\Local\Temp\ipykernel_10084\94833481.py:1: DtypeWarning: Columns (47,48,54) have mixed types. Specify dtype option on import or set low_memory=False.
  PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')


## Cruce_final

In [8]:

def cruce_predial_adicionales():
    filenames= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\*.csv")

    for path in tqdm.tqdm(filenames):
        # filename= glob.glob(path)
        puntos=pd.read_csv(path)
        puntos= prep.transform_df_to_gpd(puntos, 'LONGITUD','LATITUD', 4326)
        rev_pred=gpd.read_file(path[:path.find('_puntos.csv')] + '.shp')
        UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
        nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
        nounicos=nounicos.to_crs(3857)
        puntos=puntos.set_geometry('geometry')
        rev_pred=rev_pred.set_geometry('geometry')
        rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
        j_puntos=prep.ckdnearest(puntos.to_crs(3857), puntos[['geometry']].to_crs(3857), k=2)
        # print(nounicos, 'no unicos')
        # print(nounicos.crs, 'no unicos crs')
        # print(j_puntos, 'puntos')
        # print(j_puntos.crs,'puntos crs')
        # print(gpd.sjoin(nounicos,j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD', 'LATITUD']]))
        cruce_check= gpd.sjoin(nounicos,j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD', 'LATITUD']])
    #   cruce_check.loc[cruce_check['CLAVECATAS']=='0240127004000000'].plot()
        file_name='Salidas'
        name=path[path.rfind('\\')+1:path.rfind('_puntos')]
        path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
        # cruce_check.to_file(path12 + '\\' +name+  '_predial_final_curts_reps.shp')
        # UNICOS.to_file(path12 + '\\' +name+ '_predial_final_curts_unicos.shp')
        # final_pred= pd.concat([cruce_check, UNICOS])
        filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")
        for path2 in filename2:
            if name in path2:
                cruces_ads=gpd.read_file(path2)
                name2=path2[path2.rfind('\\'+name)+1:path2.rfind('_cruces_todos')]
            else:
                print('El archivo aun no se encuentra en la carpeta')
                break
            if cruces_ads.crs== 4326:
                cruces_ads=cruces_ads.to_crs(3857)
            elif cruces_ads.crs == 3857:
                continue
            else:
                cruces_ads.crs=3857
            
          
            ### el primero es fácil porque es directo, resultado: poli    
            primero=gpd.sjoin(UNICOS,cruces_ads)
            ## diferenciando
            temp1=cruces_ads.loc[~cruces_ads['curt'].isin(primero['curt_x'])]
            ##cruce
            segundo= gpd.sjoin(cruce_check,temp1)
            sobra_pred=cruce_check[~cruce_check.geometry.isin(segundo.geometry)]
            real_ads= temp1.loc[~temp1['curt'].isin(segundo['curt_y'])]
            sobra_pred['CAT_1']='FUERA'
            primero['CAT_1']='PREDIAL_UNICOS'
            segundo['CAT_1']= 'PREDIAL_REPETIDOS'
            pred_final= pd.concat([primero, segundo,sobra_pred])
            pred_final.dropna(how='all', axis=1, inplace=True)
            drop=pred_final.columns[(pred_final.columns.str.contains('0|TYPES|TYPE|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
            pred_final.drop(columns=drop, inplace=True)
            
            pred_final.to_file(path12 + '\\' +name2+ 'base_final_predial_corregida.shp')
            real_ads['CAT_1']= 'ADICIONALES'
            base_final=pd.concat([pred_final, real_ads])
            # base_final.to_file(path12 + '\\' +name2+ 'base_final_ads_pred.shp')
            return(base_final)

In [11]:
cruce_predial_adicionales()
base_final

NameError: name 'base_final' is not defined

In [56]:
filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")
for path2 in filename2:
    print(path2)

C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\Ixtapan_casas_cruces_todos.shp
C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\Ixtapan_terrenos_cruces_todos.shp


In [85]:
name=path[path.rfind('\\')+1:path.rfind('_puntos')]
path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
path2[path2.rfind(path12 + '\\' +name):path2.rfind('_cruces_todos')]

''

In [94]:
name

'Ixtapan'

'Ixtapan_terrenos'

In [88]:
filename2 + '\\' +name

TypeError: can only concatenate list (not "str") to list

In [ ]:
test0=pd.concat([m1,ADS_F,PRED_F, fin_casas], axis=0)
# .drop_duplicates('geometry')

In [ ]:
predial.loc[~predial['CLAVECATASTRAL'].isin(PRED['CLAVECATASTRAL'])]

### Distancias mínimas

In [ ]:
from shapely.geometry import Point, LineString
import geopandas as gpd
from geoloc2.preparacion_datos.src.preparacion_inter_puntos import ckdnearest
gdf=gpd.read_file("C:/Users/ASUS/Desktop/qgiszonas/points.shp")
gdf["LATLON"]=gdf["geometry"].x.astype("str")+","+gdf["geometry"].y.astype("str")
gdf


# usar primero sjoin_nearest y si hay mucho fallo usar la función ckdnearest

In [ ]:
casas=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\casas_t_especificios_1.shp")
casas=casas.to_crs(gdf.crs)
casas["LATLON"]=casas["geometry"].centroid.x.astype("str")+","+casas["geometry"].centroid.y.astype("str")
casas["geometry1"]=casas["geometry"]
casas["geometry"]=casas["geometry"].representative_point()
casas=casas.reset_index()
casas.rename(columns={"index":"FID"},inplace=True)
casas

In [ ]:
terreno=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\terreno_t_especificios_1.shp")
terreno=terreno.to_crs(gdf.crs)
terreno["LATLON"]=terreno["geometry"].centroid.x.astype("str")+","+terreno["geometry"].centroid.y.astype("str")
terreno["geometry1"]=terreno["geometry"]
terreno["geometry"]=terreno["geometry"].representative_point()
terreno

In [ ]:

# JFF_c = JFF_c.loc[JFF_c['min_dist'] < 10]
JFF_c

In [ ]:
JFF_terreno = ckdnearest(gdf.drop(columns='LATLON').to_crs(3857), terreno.to_crs(3857))
# JFF_terreno = JFF_terreno.loc[JFF_terreno['min_dist'] < 10]
JFF_terreno

In [ ]:
JFF_c["origen"]="Casas"
JFF_terreno["origen"]="Terreno"
concatenado=pd.concat([JFF_c,JFF_terreno],ignore_index=True)
concatenado.sort_values(by="min_dist",inplace=True)
concatenado=concatenado.drop_duplicates(subset=["CLAVECATAS"],keep="first")
concatenado

In [ ]:
concatenado[["FID","origen","CLAVECATAS","geometry"]].to_crs("3857").to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Puntos_cercanos.shp")


In [ ]:
a=concatenado[["FID","origen","CLAVECATAS","geometry1"]]
a=gpd.GeoDataFrame(a[["FID","origen","CLAVECATAS"]],geometry=a["geometry1"])
a.crs=6364
a=a.to_crs("3857")
a.to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Poligonos_cercanos.shp")

## Correcciones

In [6]:
N= pd.read_csv('E:\Downloads\INTFIS_BASE_FINAL_NAUCALPAN_por_coma\INTFIS_BASE_FINAL_NAUCALPAN_por_coma.csv', encoding='utf-8-sig')

C:\Users\dlara\AppData\Local\Temp\ipykernel_42820\3829993678.py:1: DtypeWarning: Columns (1,2,4,5,7,8,9,10,11,12,36,37,38,39,40,41,57,58,68,69,71,73,74,75,76,82,85) have mixed types. Specify dtype option on import or set low_memory=False.
  N= pd.read_csv('E:\Downloads\INTFIS_BASE_FINAL_NAUCALPAN_por_coma\INTFIS_BASE_FINAL_NAUCALPAN_por_coma.csv', encoding='utf-8-sig')


In [11]:
PRED=N.loc[N['CLAVECATASTRAL_RECAUDACION'].notna()].dropna(how='all')

In [78]:
correct_pred= PRED.loc[PRED['CLAVECATASTRAL'].astype(str).str.startswith('00')].groupby(['CLAVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO']).first().dropna(axis=1,how='all').drop(columns='Unnamed: 0').reset_index()
# correct_pred['CLAVECATASTRAL']=correct_pred['CLAVECATASTRAL'].str[-12:].str.replace('000','', regex=False).str.zfill(10)

In [107]:
correct_pred['CLAVECATASTRAL'].str[-16:]
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00000'), 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00000')]['CLAVECATASTRAL'].str[-11:].str.zfill(12).str.replace('000','', regex=False).str.zfill(10)

In [124]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('000'),'CLAVECATASTRAL']=correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('000')]['CLAVECATASTRAL'].str[-16:].str.replace('000','')

In [141]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00'),'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00')]['CLAVECATASTRAL'].str[:12].str.replace('00','').str.zfill(9)

In [147]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()<9, 'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()<9]['CLAVECATASTRAL'] + '00000000'

In [154]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==9, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==9]['CLAVECATASTRAL'] + '0000000'

In [161]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==10, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==10]['CLAVECATASTRAL'].str.zfill(10) + '000000'

In [164]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==12, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==12]['CLAVECATASTRAL'] + '0000'

In [167]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==13, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==13]['CLAVECATASTRAL'] + '000'

In [169]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==14, 'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==14]['CLAVECATASTRAL'] + '00'

In [190]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==16, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==16]['CLAVECATASTRAL'].str[:-6].str.replace('098','98').str.ljust(15,fillchar='0').str.zfill(16)

In [206]:
# PRED=PRED.loc[~PRED['CLAVECATASTRAL'].astype(str).str.startswith('00')]
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].str.replace('.0','', regex=False).str.zfill(16)

In [213]:
# PRED=PRED.dropna(axis=1,how='all').drop(columns='Unnamed: 0')
ZPRED=pd.concat([PRED,correct_pred ], axis=0)
# ZPRED.CLAVECATASTRAL.value_counts()
ZPRED.loc[ZPRED['CLAVECATASTRAL'].isna(), 'CLAVECATASTRAL'] =ZPRED.loc[ZPRED['CLAVECATASTRAL'].isna()]['CLAVECATASTRAL_RECAUDACION'].str.replace('CAT-','').astype(str)

In [233]:
duplicados=ZPRED.loc[ZPRED['CLAVECATASTRAL'].isin(ZPRED['CLAVECATASTRAL'][ZPRED['CLAVECATASTRAL'].duplicated()])].sort_values(by='CLAVECATASTRAL').groupby(['CLAVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO']).first().reset_index()

In [235]:
ZPRED=ZPRED.loc[~ZPRED['CLAVECATASTRAL'].isin(ZPRED['CLAVECATASTRAL'][ZPRED['CLAVECATASTRAL'].duplicated()])]
ZZPRED= pd.concat([ZPRED,duplicados], axis=0)

In [273]:
ZZPRED['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + ZZPRED['CLAVECATASTRAL']

In [285]:
NF=pd.read_csv(r'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv',encoding='utf-8-sig')

C:\Users\dlara\AppData\Local\Temp\ipykernel_42820\848107519.py:1: DtypeWarning: Columns (2,4,5,6,8,9,10,11,12,13,14,15,16,17,19,20,21,23,24,25,26,27,30,32,34,35,38,39,40,43,44,49,50,51,52,53,55,56,57,60,61,62,63,64,65,66,67,68,69,70,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,101,102,103,104,107,108,109,110,111,113,114,115,116,117,118,119,124,127,129,130,131,134,135,137,138,139,140,141,142,146,156,157,158,159,160,161,162,164,166,167,168,169,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,219,220) have mixed types. Specify dtype option on import or set low_memory=False.
  NF=pd.read_csv(R'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv',encoding='utf-8-sig')


In [283]:
NFS=gpd.read_file(r'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.shp')

In [300]:
PRED2=NF.loc[NF['CLAVECATASTRAL'].notna()]

In [359]:
ZZPRED.loc[ZZPRED['CLAVECATASTRAL']=='09807751100E0904']

,CLAVECATASTRAL,CLAVECATASTRAL_RECAUDACION,ORDEN,BDINTERNA_CONT_NOMBRE_COMPLETO,BDINTERNA_CONT_RFC,BDINTERNA_CONT_EMAIL,BDINTERNA_CONT_DIRECCION,BD_INTERNA_FUENTE_ORIGEN,CURT,CURT_ESTATUS,...,INTFIS_RS_FD_PLACE_ID,INTFIS_RS_FD_TYPES,INTFIS_RS_FD_LAT,INTFIS_RS_FD_LON,INTFIS_RS_FD_TIPO_VIALIDAD,INTFIS_RS_FD_PLUS_CODE,IMPORTE_FEB_MAR.1,ADEUDOS_REGISTRADOS.1,ULTIMO_VALOR_CATASTRAL.1,ETIQUETA
418343,09807751100E0904,CAT-09807751100E0904,121639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [301]:
PRED2['CLAVECATASTRAL']=PRED2['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)

C:\Users\dlara\AppData\Local\Temp\ipykernel_42820\4246564689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRED2['CLAVECATASTRAL']=PRED2['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)


In [305]:
PRED2['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + PRED2['CLAVECATASTRAL']

C:\Users\dlara\AppData\Local\Temp\ipykernel_42820\2610296744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRED2['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + PRED2['CLAVECATASTRAL']


In [346]:

# denue=PRED2.columns[PRED2.columns.str.contains('DENUE')]
# PRED2.drop(columns=denue,inplace=True)
# drop1=PRED2.columns[(~PRED2.columns.isin(ZZPRED.columns)) & (PRED2.columns.str.contains('SAT|ESTATUS|NOMINA'))]
# PRED2.drop(columns=drop1,inplace=True)

drop2=PRED2.columns[(~PRED2.columns.isin(ZZPRED.columns)) & (PRED2.columns.str.contains('N_EXT|KEY|GRUPO_CONTROL|BDINTERNA|IMSS|CALLE|COLONIA|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
PRED2.drop(columns=drop2,inplace=True)

C:\Users\dlara\AppData\Local\Temp\ipykernel_42820\706618897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PRED2.drop(columns=drop2,inplace=True)


In [357]:
PRED2[['CLAVECATASTRAL','LATLON',  'CVEGEO',
       'Tipo Polígono', 'ORIGEN', 'TIPO DE COORDENADA', 'CURT_NOTAS', 'IMPORTE_FEB',
       'IMPORTE_MAR', 'ACTIVO', 'ACTIVIDAD_ECONOMICA', 'NUM_TRABAJADORES',
       'RANGO_TRABAJADORES', 'CLASIFICACION_TRABAJADORES', 'ACTIVO2',
       'CANTIDAD_ANIOS_ADEUDOS', 'ZONA_CATASTRAL', 'IMPORTE_FEB_X_ANIO',
       'CLASIFICACION_VALORCATASTRAL', 'USO', 'INTFIS_RS_FD_COOR_RECAUDACION',
       'INTFIS_RS_FD_ESTADO', 'INTFIS_RS_FD_PAIS', 'INTFIS_RS_FD_URLS',
       'CONSECUTIVO', 'RFC', 'RFC_CONTEOS', 'INTFIS_RS_FD_FULL_DOM', 'ID_CAT',
       'curt', 'DOMICILIO', 'NUM_EXT', 'NUM_INT', 'CP', 'CONST_DOM',
       'DOM_UNICOS', 'PERSONA', 'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009',
       'VALORCATASTRAL_2010', 'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012',
       'VALORCATASTRAL_2013', 'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015',
       'PREDIAL_PROPIETARIO_EMAIL', 'PREDIAL_PROPIETARIO_CURP',
       'PREDIAL_PROPIETARIO_RFC', 'IMPORTETOTAL_ACUMULATIVO_AGUA',
       'CANTIDAD_NIS', 'NIS', 'CANTIDAD_CLAVES_CATASTRALES',
       'CLAVES_CATASTRALES', 'INTFIS_RS_FD_LATLON','curt','CURT','INTFIS_RS_FD_LON', 'INTFIS_RS_FD_LAT','INTFIS_RS_FD_LON_1', 'INTFIS_RS_FD_LAT_1','INTFIS_RS_FD_LAT_2', 'INTFIS_RS_FD_LON_2', '0', 'geometry']]

,CLAVECATASTRAL,LATLON,CVEGEO,Tipo Polígono,ORIGEN,TIPO DE COORDENADA,CURT_NOTAS,IMPORTE_FEB,IMPORTE_MAR,ACTIVO,...,curt,CURT,INTFIS_RS_FD_LON,INTFIS_RS_FD_LAT,INTFIS_RS_FD_LON_1,INTFIS_RS_FD_LAT_1,INTFIS_RS_FD_LAT_2,INTFIS_RS_FD_LON_2,0,geometry
25144,09807751100E0904,"19.486213207550765 ,-99.24445849052132",150570001018A010,NaN,NPRED2,INTERPOLADA,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.244458,19.486213,NaN,NaN,19.486255,-99.244405,NaN,POINT (-99.24445849052132 19.486213207550765)
25145,09807751100F0102,"19.48615974843129 ,-99.24451698108732",150570001018A010,NaN,NPRED2,INTERPOLADA,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.244517,19.486160,NaN,NaN,19.486214,-99.244516,NaN,POINT (-99.24451698108732 19.48615974843129)
25146,09807751100F0203,"19.486106289311767 ,-99.2445754716534",150570001018A010,NaN,NPRED2,INTERPOLADA,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.244575,19.486106,NaN,NaN,19.486196,-99.244555,NaN,POINT (-99.2445754716534 19.486106289311767)
25147,09807751100F0304,"19.486052830192293 ,-99.2446339622194",150570001018A010,NaN,NPRED2,INTERPOLADA,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.244634,19.486053,NaN,NaN,19.486121,-99.244458,NaN,POINT (-99.2446339622194 19.486052830192293)
25148,09807751100F0403,"19.485999371072776 ,-99.24469245278549",150570001018A010,NaN,NPRED2,INTERPOLADA,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.244692,19.485999,NaN,NaN,19.485899,-99.244739,NaN,POINT (-99.24469245278549 19.485999371072776)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321411,0981896206000000,"19.451674 ,-99.2665824",NaN,NaN,NPRED2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.266582,19.451674,NaN,NaN,NaN,NaN,NaN,POINT (-99.2665824 19.451674)
321412,0981896304000000,"19.4515 ,-99.2665",NaN,NaN,BPC2_1,NaN,NaN,4994.0,4645.0,NaN,...,NaN,192705225309915597430,-99.266500,19.451500,NaN,NaN,NaN,NaN,NaN,POINT (-99.2665 19.4515)
321413,0981896305000000,"19.4514 ,-99.2664",NaN,NaN,BPC2_1,NaN,NaN,NaN,3907.0,NO,...,192705308309915590778,192705059809915595083,-99.266400,19.451400,NaN,NaN,NaN,NaN,NaN,POINT (-99.2664 19.4514)
321414,0981898402000000,"19.4489309 ,-99.2698127",NaN,NaN,NPRED2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-99.269813,19.448931,NaN,NaN,NaN,NaN,NaN,POINT (-99.2698127 19.4489309)


In [311]:
ZZPRED.loc[~ZZPRED['CLAVECATASTRAL'].isin(PRED2['CLAVECATASTRAL'])]
# PRED2.loc[~PRED2['CLAVECATASTRAL'].isin(ZZPRED['CLAVECATASTRAL'])]

,CLAVECATASTRAL,CLAVECATASTRAL_RECAUDACION,ORDEN,BDINTERNA_CONT_NOMBRE_COMPLETO,BDINTERNA_CONT_RFC,BDINTERNA_CONT_EMAIL,BDINTERNA_CONT_DIRECCION,BD_INTERNA_FUENTE_ORIGEN,CURT,CURT_ESTATUS,...,INTFIS_RS_FD_PLACE_ID,INTFIS_RS_FD_TYPES,INTFIS_RS_FD_LAT,INTFIS_RS_FD_LON,INTFIS_RS_FD_TIPO_VIALIDAD,INTFIS_RS_FD_PLUS_CODE,IMPORTE_FEB_MAR.1,ADEUDOS_REGISTRADOS.1,ULTIMO_VALOR_CATASTRAL.1,ETIQUETA
286600,0980903636000000,CAT-0980903636000000,1184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ChIJ5YtZSGcF0oURCtlSLx4qZIg,"('car_repair', 'establishment', 'point_of_inte...",19.443588,-99.282422,Calle,"CPV9+C2 Naucalpan de Juárez, State of Mexico, ...",NaN,NaN,NaN,NaN
286636,0980441101000000,CAT-0980441101000000,38569,TREJO BAÑOS SERGIO,NaN,NaN,VERACRUZ 17 SAN ANTONIO ZOMEYUCAN S/N -,PREDIAL_MUNICIPAL,192709233109914532535,Activo,...,ChIJTyjK9LgD0oURP3r0adIzEzA,"('s', 't', 'r', 'e', 'e', 't', '_', 'a', 'd', ...",19.452664,-99.248190,Calle,"FQ32+3P Naucalpan de Juárez, State of Mexico, ...",NaN,NaN,NaN,NaN
286637,0980422501000000,CAT-0980422501000000,39452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ChIJi67bbIwD0oURjbLx13zYvAs,"('street_address',)",19.454231,-99.272486,NaN,"FP3H+M2 Naucalpan de Juárez, State of Mexico, ...",NaN,NaN,NaN,NaN
286645,0981098202000000,CAT-0981098202000000,43379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ChIJIbSSpHUE0oURU0K6v99zW4Q,"('s', 't', 'r', 'e', 'e', 't', '_', 'a', 'd', ...",19.453081,-99.280364,Calle,"FP39+6V Naucalpan de Juárez, State of Mexico, ...",NaN,NaN,NaN,NaN
286649,0980959111000000,CAT-0980959111000000,45039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ChIJDRHfVg4E0oURqgcoIseQzgw,"('s', 't', 'r', 'e', 'e', 't', '_', 'a', 'd', ...",19.440815,-99.287168,Calle,"CPR7+84 Naucalpan de Juárez, State of Mexico, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,0980944508000000,CAT-0980944508000000,3547,ALONSO DAMIAN JOSE COSME,None,None,"10 DE ABRIL LT 7, MZ.100 S/N -",PREDIAL_MUNICIPAL,192608408909915588376,Activo,...,ChIJ4USVIeUD0oURePqVjCUhlz4,['street_address'],19.435791,-99.266136,ROOFTOP,"CPPM+8G Naucalpan de Juárez, State of Mexico, ...",NaN,None,NaN,None
287,0980944508000000,CAT-0980944508000000,439238,ESPINOZA GARCIA ARMANDO,None,None,10 DE ABRIL LT 9 MZ 45 S/N 0,PREDIAL_MUNICIPAL,None,Activo,...,None,None,NaN,NaN,None,None,NaN,None,NaN,None
297,0980948528000000,CAT-0980948528000000,439242,MERCADO ORDOÑEZ LIDIA,None,None,C/DOROTEA MZ 9 LT 10 LA MANCHA 11 S/N 0,PREDIAL_MUNICIPAL,None,Activo,...,None,None,NaN,NaN,None,None,NaN,None,NaN,None
298,0980948528000000,CAT-0980948528000000,3638,SANTIAGO VAZQUEZ ELPIDIO,None,None,C TRISTE FIGUERA LT 28 MZ 9 M II S/N 0,PREDIAL_MUNICIPAL,192554164409916177316,Activo,...,ChIJoTAlcvwD0oURF-S4fGP0H4M,['street_address'],19.431726,-99.271367,ROOFTOP,"CPJH+MF Naucalpan de Juárez, State of Mexico, ...",NaN,None,NaN,None
